In [1]:
import torch
from torch_geometric.data import Data
from torch.utils.data import Dataset
from torch import optim
from torch.nn.functional import softmax
from torch.utils.data import DataLoader
import argparse
import numpy as np
from data import *
from ABGCN import *
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import f1_score
from utils import *
from random import randint, shuffle
from tqdm import tqdm
from copy import copy
import sys

print('preparing data...', end='')
dataset = semEval2017Dataset(dataPath = '../dataset/semeval2017-task8/', 
                                type = 'test',
                                w2vPath = '../dataset/glove/',
                                w2vDim = 100)
loader = DataLoader(dataset, shuffle=True)
with open('../dataset/semeval2017-task8/' + 'testSet.json', 'r') as f:
        content = f.read()
rawDataset = json.loads(content)
label2IndexRumor = copy(rawDataset['label2IndexRumor'])
label2IndexStance = copy(rawDataset['label2IndexStance'])
del rawDataset


preparing data...

In [2]:
device = torch.device('cpu')
model = ABGCN(w2vDim = 100,
              s2vDim = 128,
              gcnHiddenDim = 128,
              rumorFeatureDim = 128,
              numRumorTag = len(label2IndexRumor),
              numStanceTag = len(label2IndexStance),
              s2vMethon = 'l')
model = model.set_device(device)
model.load('./model/model.pt')
loss_func = torch.nn.CrossEntropyLoss(reduction='mean').to(device)
rumorTrue = []
rumorPre = []
totalLoss = 0.

model.eval()
for data in tqdm(iter(loader), desc="[epoch {:d}, rumor]".format(1), leave=False, ncols=100):
    # 抹除dataloader生成batch时对数据的升维
    data['threadId'] = data['threadId'][0]
    data['threadIndex'] = data['threadIndex'][0]
    for i in range(len(data['nodeFeature'])):
        data['nodeFeature'][i] = data['nodeFeature'][i].view((data['nodeFeature'][i].shape[1], data['nodeFeature'][i].shape[2]))
    data['edgeIndexTD'] = data['edgeIndexTD'].view((data['edgeIndexTD'].shape[1], data['edgeIndexTD'].shape[2]))
    data['edgeIndexBU'] = data['edgeIndexBU'].view((data['edgeIndexBU'].shape[1], data['edgeIndexBU'].shape[2]))
    data['rumorTag'] = data['rumorTag'].view((data['rumorTag'].shape[1]))
    data['stanceTag'] = data['stanceTag'].view((data['stanceTag'].shape[1]))
    rumorTag = data['rumorTag'].to(device)
    rumorTrue += data['rumorTag'].tolist()

    p = model.forwardRumor(data)
    loss = loss_func(p, rumorTag)
    totalLoss += loss
    
    p = softmax(p, dim=1)
    rumorPre += p.max(dim=1)[1].tolist()
microF1Rumor = f1_score(rumorTrue, rumorPre, labels=[0,1,2], average='micro')
macroF1Rumor = f1_score(rumorTrue, rumorPre, labels=[0,1,2], average='macro')
accuracyRumor = (np.array(rumorTrue) == np.array(rumorPre)).sum() / len(rumorPre)
print('rumor detection:')
print('average loss: {:f}, accuracy: {:f}, micro-f1: {:f}, macro-f1: {:f}'.format(
    totalLoss / len(loader), accuracyRumor, microF1Rumor, macroF1Rumor
))

rumor detection:
average loss: 0.188951, accuracy: 1.000000, micro-f1: 1.000000, macro-f1: 1.000000
